# ONNX Runtime Inference

This tutorial demonstrates how to run inference using an exported ONNX model with onnxruntime.

In [ ]:
import cv2
import numpy as np
import torch
import onnxruntime as ort
import matplotlib.pyplot as plt
from modern_yolonas.inference.preprocess import preprocess
from modern_yolonas.inference.postprocess import postprocess, rescale_boxes
from modern_yolonas.inference.visualize import draw_detections, COCO_NAMES

# Load ONNX model
onnx_path = "model_float32.onnx"  # Update with your model path
session = ort.InferenceSession(onnx_path)

# Load a test image
test_image_path = "test.jpg"  # Update with your image path
original = cv2.imread(test_image_path)
orig_h, orig_w = original.shape[:2]

# Preprocess
input_tensor, pad_info = preprocess(original, input_size=640)

# Run inference
pred_bboxes, pred_scores = session.run(None, {"images": input_tensor.numpy()})

# Postprocess (NMS)
boxes, scores, class_ids = postprocess(
    torch.from_numpy(pred_bboxes),
    torch.from_numpy(pred_scores),
    conf_threshold=0.25,
    iou_threshold=0.45,
)

# Rescale boxes to original image size
if len(boxes) > 0:
    boxes = rescale_boxes(boxes, pad_info, (orig_h, orig_w))

print(f"Detections: {len(boxes)}")

In [ ]:
# Visualize
class_names = list(COCO_NAMES)  # Replace with your class names if custom-trained
annotated = draw_detections(
    original, boxes.numpy(), scores.numpy(), class_ids.numpy(),
    class_names=class_names,
)

plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
plt.title(f"ONNX Runtime Inference ({len(boxes)} detections)")
plt.axis("off")
plt.show()

print(f"\nDetections:")
for i in range(len(boxes)):
    cls_name = class_names[int(class_ids[i])] if int(class_ids[i]) < len(class_names) else f"class_{int(class_ids[i])}"
    print(f"  {cls_name}: {scores[i]:.2f}")